In [ ]:
import os
from pathlib import Path
from gurobipy import Model, GRB
from data_loader import load_prices, load_storage, load_efficiency, load_plant_capacity, load_demand
import pandas as pd
import numpy as np

# Data Loading from data folder in the parent directory
current_dir = Path().resolve()
data_folder = os.path.join(current_dir.parent, "data")
fuel_prices_file = os.path.join(data_folder, "fuel_prices.csv")           # Price in EUR/tonne (coal,oil) or EUR/m3 (gas)
storage_file = os.path.join(data_folder, "storage.csv")         # Max storage in tonnes/m3
efficiency_file = os.path.join(data_folder, "efficiency.csv")   # efficiency of MWh th to MWh el
plant_file = os.path.join(data_folder, "plant_capacity.csv")    # Plant capacity in MWh/month
demand_file = os.path.join(data_folder, "electricity_demand.csv")           # Monthly demand in MWh

start_date = "2020-01"
end_date = "2020-12"

fuels = ["coal", "oil", "gas"]
zone = 'DK_2'

try:
    fuel_prices_df = load_prices(fuel_prices_file, start_date, end_date)
    storage = load_storage(storage_file)
    efficiency = load_efficiency(efficiency_file)
    plant_capacity = load_plant_capacity(plant_file)
    demand_df = load_demand(demand_file, start_date, end_date, zone=zone ,supply_factor=0.0005)
except Exception as e:
    raise RuntimeError(f"Error loading data: {e}")


# Timesteps
T = list(range(len(demand_df)))

# Model Initialization

m = Model("Fuel_Procurement_OneMonth")

F = [fuel for fuel in fuel_prices_df.columns if fuel in fuels]

# Decision Variables
# Decision Variables (corrected filters)
x = {(fuel, t): m.addVar(lb=0, ub=storage[fuel], name=f"buy_{fuel}_{t}")
     for fuel in F for t in T}

y = {(fuel, t): m.addVar(lb=0, ub=plant_capacity[fuel], name=f"gen_{fuel}_{t}")
     for fuel in F for t in T}

s = {(fuel, t): m.addVar(lb=0, ub=storage[fuel], name=f"s_{fuel}_{t}")
     for fuel in F for t in T}




m.update()



In [ ]:
# Initial Storage Levels
initial_storage = {fuel: 0 for fuel in fuel_prices_df.columns if fuel in fuels}

# Constraints

# Demand constraint: one per time period
for t in T:
    m.addConstr(
        sum(y[fuel, t] for fuel in F) >= int(demand_df.iloc[t][zone]),
        name=f"Demand_Constraint_{t}"
    )

# efficiency constraints
for fuel in F:
    for t in T:
        m.addConstr(
            y[fuel, t] <= efficiency[fuel] * x[fuel, t],
            name=f"Efficiency_Constraint_{fuel}_{t}"
        )

m.update()

# State-of-energy (SOE) constraints
for fuel in F:
    print(f"Setting up SOE constraints for {fuel}")
    for t in T:
        print(f"  Time period {t}")
        if t == 0:
            m.addConstr(
                s[fuel, t] == initial_storage[fuel] + x[fuel, t] - y[fuel, t] / efficiency[fuel],
                name=f"SOE_Def_{fuel}_{t}"
            )
        else:
            m.addConstr(
                s[fuel, t] == s[fuel, t-1] + x[fuel, t] - y[fuel, t] / efficiency[fuel],
                name=f"SOE_Def_{fuel}_{t}"
            )

# Storage limits
for fuel in F:
    for t in T:
        m.addConstr(
            s[fuel, t] <= storage[fuel],
            name=f"SOE_Limit_{fuel}_{t}"
        )

# Cyclic SOE (start == end)
t0 = 0
t4 = T[-1]   # last period
for fuel in F:
    m.addConstr(
        s[fuel, t0] == s[fuel, t4],
        name=f"SOE_Cycle_{fuel}"
    )

# Objective
m.setObjective(
    sum(fuel_prices_df.iloc[t][fuel] * x[fuel, t] for fuel in F for t in T),
    GRB.MINIMIZE
)
m.update()

# Optimize
m.optimize()

# print results
if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    # Purchased and generated per fuel and period
    for (fuel, t) in sorted(x.keys()):
        print(f"Purchased {x[fuel, t].X:.4f} units of {fuel} at period {t}")
    for (fuel, t) in sorted(y.keys()):
        print(f"Generated {y[fuel, t].X:.4f} MWh from {fuel} at period {t}")

    # Cost per fuel aggregated across periods
    total_cost = 0.0
    for fuel in F:
        cost_f = sum(fuel_prices_df.iloc[t][fuel] * x[fuel, t].X for t in T)
        print(f"Cost for {fuel}: EUR {cost_f:.2f}")
        total_cost += cost_f

    print(f"Total Cost: EUR {total_cost:.2f}")
else:
    print("No optimal solution found.")


Setting up SOE constraints for coal
  Time period 0
  Time period 1
  Time period 2
  Time period 3
  Time period 4
  Time period 5
  Time period 6
  Time period 7
  Time period 8
  Time period 9
  Time period 10
  Time period 11
Setting up SOE constraints for oil
  Time period 0
  Time period 1
  Time period 2
  Time period 3
  Time period 4
  Time period 5
  Time period 6
  Time period 7
  Time period 8
  Time period 9
  Time period 10
  Time period 11
Setting up SOE constraints for gas
  Time period 0
  Time period 1
  Time period 2
  Time period 3
  Time period 4
  Time period 5
  Time period 6
  Time period 7
  Time period 8
  Time period 9
  Time period 10
  Time period 11
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 123 rows, 108 columns and 291 nonzeros
Model fingerprint: 0x6bafad90
Coefficient statistics:
  Matrix ran

In [10]:
# 1) Scenario definitions
scenario_list = ["low", "standard", "high"]

# 2) Multiplicative factors per fuel and scenario
#    - Gas is most volatile, coal least volatile
#    - Factors are multiplicative: scaled_price = base_price * factor
price_factors = {
    "coal":    {"low": 0.97, "standard": 1.00, "high": 1.03},   # +/- ~3%
    "oil":     {"low": 0.92, "standard": 1.00, "high": 1.08},   # ~8% swing
    "gas":     {"low": 0.80, "standard": 1.00, "high": 1.25},   # large volatility
}

# If you have extra fuels (e.g. 'lignite') present in fuel_prices_df, we preserve them (standard=1.0)
for col in fuel_prices_df.columns:
    if col not in price_factors:
        price_factors[col] = {s: 1.0 for s in scenario_list}

# 3) Seasonal probabilities (recommended)
#    Rationale: fuels (especially gas) tend to show seasonal patterns (higher risk of high prices in winter).
#    Seasonal buckets: winter=Dec/Jan/Feb, spring=Mar/Apr/May, summer=Jun/Jul/Aug, autumn=Sep/Oct/Nov
seasonal_scenario_probs = {
    "winter": {"low": 0.10, "standard": 0.40, "high": 0.50},
    "spring": {"low": 0.25, "standard": 0.60, "high": 0.15},
    "summer": {"low": 0.40, "standard": 0.50, "high": 0.10},
    "autumn": {"low": 0.20, "standard": 0.60, "high": 0.20},
}

def month_to_season(month: int) -> str:
    if month in (12, 1, 2):
        return "winter"
    if month in (3, 4, 5):
        return "spring"
    if month in (6, 7, 8):
        return "summer"
    return "autumn"

# 4) Build scenario_prices (dict of DataFrames) and scenario probability DataFrame (indexed by original index)
scenario_prices = {s: fuel_prices_df.copy(deep=True) for s in scenario_list}

# Ensure index is datetime (should be already from your loader) and use it to create probabilities by period
index = fuel_prices_df.index
if not isinstance(index, pd.DatetimeIndex):
    # fallback: if index was integer-range, try to use a 'month' column if present
    if "month" in fuel_prices_df.columns:
        index = pd.to_datetime(fuel_prices_df["month"])
        for df in scenario_prices.values():
            df.index = index
    else:
        raise RuntimeError("fuel_prices_df must have a DatetimeIndex or a 'month' column parseable to dates.")

# Create a DataFrame of probabilities for each period
scenario_prob_df = pd.DataFrame(index=index, columns=scenario_list, dtype=float)

for ts in index:
    season = month_to_season(ts.month)
    probs = seasonal_scenario_probs[season]
    # assign same probabilities for all fuels in that month (you could vary by fuel if desired)
    scenario_prob_df.loc[ts, list(probs.keys())] = [probs[k] for k in scenario_list]

# 5) Create scaled price DataFrames
for s in scenario_list:
    df_scaled = fuel_prices_df.copy(deep=True)
    # apply fuel-specific factor for each column
    for fuel in df_scaled.columns:
        factor = price_factors.get(fuel, {}).get(s, 1.0)
        # multiply only numeric values
        df_scaled[fuel] = pd.to_numeric(df_scaled[fuel], errors="coerce") * factor
    scenario_prices[s] = df_scaled

# 6) Some handy objects for the modelling step later
scenario_index_to_t = {i: s for i, s in enumerate(scenario_list)}  # e.g. 0->"low"
# scenario probabilities per time-step as numpy array shaped (T, n_scenarios)
prob_array = scenario_prob_df[scenario_list].to_numpy(dtype=float)

# 7) Quick sanity prints / checks
print("Scenarios:", scenario_list)
print("Price factors (per fuel):")
for f, fac in price_factors.items():
    print(f"  {f}: {fac}")
print()
print("Sample of scenario probabilities (first 6 periods):")
display(scenario_prob_df.head(6))

print("Sample scaled prices (first 3 fuels, first 5 rows) for each scenario:")
for s in scenario_list:
    print(f"\n--- {s.upper()} scenario ---")
    display(scenario_prices[s].iloc[:, :3].head(5))


# - scenario_prices: dict {scenario_name: DataFrame_of_scaled_prices}
# - scenario_prob_df: DataFrame indexed by time with columns ['low','standard','high'] giving probabilities
# - prob_array: numpy array (T x 3) with probabilities for building the stochastic model
# - scenario_list and scenario_index_to_t helpers



Scenarios: ['low', 'standard', 'high']
Price factors (per fuel):
  coal: {'low': 0.97, 'standard': 1.0, 'high': 1.03}
  oil: {'low': 0.92, 'standard': 1.0, 'high': 1.08}
  gas: {'low': 0.8, 'standard': 1.0, 'high': 1.25}
  lignite: {'low': 1.0, 'standard': 1.0, 'high': 1.0}

Sample of scenario probabilities (first 6 periods):


,low,standard,high
Unnamed: 0,,,
2020-01-31,0.10,0.4,0.50
2020-02-29,0.10,0.4,0.50
2020-03-31,0.25,0.6,0.15
2020-04-30,0.25,0.6,0.15
2020-05-31,0.25,0.6,0.15
2020-06-30,0.40,0.5,0.10


Sample scaled prices (first 3 fuels, first 5 rows) for each scenario:

--- LOW scenario ---


,coal,oil,lignite
Unnamed: 0,,,
2020-01-31,5.699738,43.458809,5.236369
2020-02-29,5.405407,40.300435,4.965967
2020-03-31,5.250431,32.777376,4.823589
2020-04-30,4.993639,24.501955,4.587673
2020-05-31,4.332887,22.033804,3.980638



--- STANDARD scenario ---


,coal,oil,lignite
Unnamed: 0,,,
2020-01-31,5.876019,47.237836,5.236369
2020-02-29,5.572585,43.804821,4.965967
2020-03-31,5.412815,35.627583,4.823589
2020-04-30,5.148081,26.632560,4.587673
2020-05-31,4.466893,23.949786,3.980638



--- HIGH scenario ---


,coal,oil,lignite
Unnamed: 0,,,
2020-01-31,6.052299,51.016863,5.236369
2020-02-29,5.739762,47.309207,4.965967
2020-03-31,5.575200,38.477790,4.823589
2020-04-30,5.302523,28.763164,4.587673
2020-05-31,4.600900,25.865769,3.980638


In [ ]:
# 0) sanity checks
assert "scenario_list" in globals(), "scenario_list not found. Run the stochastic input cell first."
assert "scenario_prices" in globals(), "scenario_prices not found. Run the stochastic input cell first."

# 1) Build scenario probabilities (simple time-average of per-period probs)
#    if scenario_prob_df exists we average across time; otherwise fall back to equal probs
if 'scenario_prob_df' in globals():
    # ensure columns match scenario_list
    prob_series = scenario_prob_df[scenario_list].mean(axis=0)
    # normalize to sum 1 (guard against rounding)
    prob_vals = prob_series.to_numpy(dtype=float)
    if prob_vals.sum() <= 0:
        # fallback to equal probabilities
        prob_vals = np.ones(len(scenario_list)) / len(scenario_list)
    else:
        prob_vals = prob_vals / prob_vals.sum()
else:
    prob_vals = np.ones(len(scenario_list)) / len(scenario_list)

scenario_probs = {s: float(prob_vals[i]) for i, s in enumerate(scenario_list)}
print("Using scenario probabilities (time-averaged):", scenario_probs)

# 2) Create extensive form model
m_stoch = Model("Fuel_Procurement_Stochastic_ExtensiveForm")

# decision variables: scenario-indexed copies
# x_s[(s,f,t)] purchase quantity for fuel f at time t under scenario s
# y_s[(s,f,t)] generation for fuel f at time t under scenario s
# s_s[(s,f,t)] storage level
x_s = {}
y_s = {}
soe_s = {}

for s in scenario_list:
    for f in F:
        for t in T:
            # bounds: purchases and storage and generation use same upper bounds as before
            x_s[(s, f, t)] = m_stoch.addVar(lb=0, ub=storage[f], name=f"x_{s}_{f}_{t}")
            y_s[(s, f, t)] = m_stoch.addVar(lb=0, ub=plant_capacity[f], name=f"y_{s}_{f}_{t}")
            soe_s[(s, f, t)] = m_stoch.addVar(lb=0, ub=storage[f], name=f"s_{s}_{f}_{t}")
m_stoch.update()

# 3) Non-anticipativity constraints (here: enforce first-period decisions equal across scenarios)
#    You can extend this by enforcing equality for t in some prefix if desired.
if len(scenario_list) > 1:
    base_s = scenario_list[0]
    for f in F:
        for t in [0]:  # only t=0 non-anticipativity; change to `range(k+1)` to enforce up to period k
            for s in scenario_list[1:]:
                m_stoch.addConstr(x_s[(base_s, f, t)] == x_s[(s, f, t)], name=f"NA_x_{f}_{t}_{s}")
                m_stoch.addConstr(y_s[(base_s, f, t)] == y_s[(s, f, t)], name=f"NA_y_{f}_{t}_{s}")
                m_stoch.addConstr(soe_s[(base_s, f, t)] == soe_s[(s, f, t)], name=f"NA_s_{f}_{t}_{s}")

# 4) Scenario-wise constraints (demand, efficiency, SOE, storage limits, cyclic SOE)
for s in scenario_list:
    price_df = scenario_prices[s]
    # demand: use demand_df (deterministic in this example). If demand should be stochastic, pass scenario-specific demand.
    for t in T:
        # demand constraint (per scenario)
        m_stoch.addConstr(
            sum(y_s[(s, f, t)] for f in F) >= float(demand_df.iloc[t][zone]),
            name=f"Demand_{s}_{t}"
        )

    # efficiency constraints and SOE
    for f in F:
        for t in T:
            # efficiency constraint: generation <= efficiency * purchase
            m_stoch.addConstr(
                y_s[(s, f, t)] <= efficiency[f] * x_s[(s, f, t)],
                name=f"Conv_{s}_{f}_{t}"
            )

            # SOE dynamic
            if t == 0:
                m_stoch.addConstr(
                    soe_s[(s, f, t)] == initial_storage[f] + x_s[(s, f, t)] - y_s[(s, f, t)] / efficiency[f],
                    name=f"SOE_def_{s}_{f}_{t}"
                )
            else:
                m_stoch.addConstr(
                    soe_s[(s, f, t)] == soe_s[(s, f, t-1)] + x_s[(s, f, t)] - y_s[(s, f, t)] / efficiency[f],
                    name=f"SOE_def_{s}_{f}_{t}"
                )

            # storage limit (upper bound already enforced on var but keep explicit constraint for readability)
            m_stoch.addConstr(
                soe_s[(s, f, t)] <= storage[f],
                name=f"SOE_lim_{s}_{f}_{t}"
            )

        # cyclic SOE per scenario
        t0 = 0
        tlast = T[-1]
        m_stoch.addConstr(
            soe_s[(s, f, t0)] == soe_s[(s, f, tlast)],
            name=f"SOE_cycle_{s}_{f}"
        )

# 5) Objective: expected cost across scenarios
#    expected_cost = sum_s prob_s * (sum_t sum_f price_s[t,f] * x_s[s,f,t])
expected_cost_expr = 0.0
for i, s in enumerate(scenario_list):
    prob = scenario_probs[s]
    price_df = scenario_prices[s]
    for f in F:
        for t in T:
            # handle missing/NaN prices gracefully by treating them as zero (or you can raise)
            p = price_df.iloc[t].get(f, np.nan)
            if pd.isna(p):
                p = 0.0
            expected_cost_expr += prob * p * x_s[(s, f, t)]

m_stoch.setObjective(expected_cost_expr, GRB.MINIMIZE)
m_stoch.update()

# 6) Optimize
m_stoch.params.OutputFlag = 1
m_stoch.optimize()

# 7) Present expected cost and a small sample of decisions (if optimal)
if m_stoch.status == GRB.OPTIMAL:
    print("\n--- STOCHASTIC MODEL: Optimal solution found ---")
    print(f"Expected Cost (objective): {m_stoch.objVal:.2f} EUR")
    # print first-period (t=0) purchases (these are non-anticipative and equal across scenarios)
    print("\nFirst-period purchases (t=0) -- identical across scenarios due to NA constraints:")
    for f in F:
        val = x_s[(scenario_list[0], f, 0)].X
        print(f"  {f}: {val:.4f}")
    # show scenario-dependent generation example for t=0..min(2,last)
    print("\nScenario-dependent generation (first 3 periods) sample:")
    for s in scenario_list:
        print(f"\nScenario: {s} (prob {scenario_probs[s]:.3f})")
        for t in T[: min(3, len(T))]:
            gen_sum = sum(y_s[(s, f, t)].X for f in F)
            print(f"  t={t}: total generation = {gen_sum:.4f} MWh")
else:
    print("Stochastic model did not return an optimal solution; status:", m_stoch.status)

Using scenario probabilities (time-averaged): {'low': 0.23750000000000007, 'standard': 0.5249999999999999, 'high': 0.23750000000000002}
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 387 rows, 324 columns and 909 nonzeros
Model fingerprint: 0xc8023812
Coefficient statistics:
  Matrix range     [9e-02, 1e+01]
  Objective range  [9e-01, 2e+01]
  Bounds range     [4e+02, 5e+03]
  RHS range        [6e-01, 5e+03]
Presolve removed 149 rows and 27 columns
Presolve time: 0.00s
Presolved: 238 rows, 297 columns, 711 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.044125e+03   0.000000e+00      0s
     178    7.7340799e+03   0.000000e+00   0.000000e+00      0s

Solved in 178 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.73407990